# sparkle_motion — Colab / Drive setup
This notebook contains the recommended Colab setup steps and a small smoke-test harness for running the orchestrator in a GPU environment.
- Use the cells below to mount Google Drive (Colab only).
- Use the installation cell to install the ML stack from `requirements-ml.txt` (optional; heavy).
- The smoke-test cell is a safe stub that checks imports and shows how to run the orchestrator in simulation mode.

## Notes and expectations
- This notebook is intended for Google Colab (A100) runs. If you are running locally, skip the Drive mount and run the commands in a terminal.
- Before installing the heavy ML dependencies, ensure you have sufficient disk and GPU (Colab or a VM). The requirements are listed in `requirements-ml.txt`.

## Configure workspace inputs
Set these before running the helper so it knows where to create directories and which model snapshots to pull. Provide one or more repo IDs via `HF_MODELS`; set `DRY_RUN = True` or leave the list empty to skip downloads.

In [ ]:
# Cell 0: Workspace configuration (edit these as needed)
from pathlib import Path

WORKSPACE_NAME = "SparkleMotion"          # Folder created under MyDrive/
HF_MODELS = [
    "stabilityai/stable-diffusion-xl-base-1.0",
]
DRY_RUN = False                              # True = skip download/smoke actions
MOUNT_POINT = "/content/drive"             # Default Colab mount
REPO_ROOT = Path.cwd()                       # Assumes notebook is opened from repo root

print(f"Configured workspace '{WORKSPACE_NAME}' (repo root: {REPO_ROOT})")
print(f"Models to manage: {HF_MODELS or '[none specified]'}")

In [ ]:
# Cell 1: Mount Google Drive (Colab-only)
try:
    from google.colab import drive
    print('Mounting Google Drive at /content/drive...')
    drive.mount('/content/drive')
except Exception as e:
    print('Not running in Google Colab or google.colab is unavailable:', e)
    print('If you want to use Drive, mount it manually or run in Colab.')


In [ ]:
# Cell 2: Install ML dependencies from requirements-ml.txt (Colab / heavy)
import importlib.util
import subprocess


def sh(cmd):
    print('Running:', cmd)
    return subprocess.check_call(cmd, shell=True)


if importlib.util.find_spec('google.colab'):
    print('Detected Colab.\n')
    print('If you need a CUDA-optimized torch wheel, install it first as recommended in the repo notebook.\n')
    # The repository contains requirements-ml.txt at the repo root.
    # If you placed the repository under Drive, adjust the path accordingly (e.g. /content/drive/MyDrive/sparkle_motion/requirements-ml.txt).
    req_path = 'requirements-ml.txt'
    try:
        sh(f'pip install -r "{req_path}"')
    except subprocess.CalledProcessError as exc:
        print('pip install failed:', exc)
else:
    print('Not running in Colab — to install locally run:\n    pip install -r requirements-ml.txt')


## Prepare Drive workspace and download models
Use the helper script added to the repo (`scripts/colab_drive_setup.py`) to create Drive folders, optionally download Hugging Face weights, and write a smoke artifact in `outputs/colab_smoke.json`.

In [ ]:
# Cell 4: Invoke Drive helper (creates folders, optional download)
import importlib.util
import subprocess
from pathlib import Path

helper_path = REPO_ROOT / "scripts" / "colab_drive_setup.py"
if not helper_path.exists():
    print(f"Helper script not found at {helper_path}. Ensure you're running the notebook from the repo root.")
else:
    in_colab = importlib.util.find_spec("google.colab") is not None
    if not in_colab:
        print("Not running inside Google Colab. Run the helper manually from a terminal if desired:")
        model_flags = " ".join(f"--model {repo}" for repo in HF_MODELS)
        print(f"  python {helper_path} {WORKSPACE_NAME} {model_flags}")
    else:
        cmd = [
            "python",
            str(helper_path),
            WORKSPACE_NAME,
            "--mount-point",
            str(MOUNT_POINT),
        ]
        for repo_id in HF_MODELS:
            cmd.extend(["--model", repo_id])
        if DRY_RUN:
            cmd.append("--dry-run")
        print("Running helper:", " ".join(cmd))
        subprocess.check_call(cmd)

In [ ]:
# Cell 4b: Inspect smoke artifact with per-model status
import json
from pathlib import Path

smoke_path = Path(MOUNT_POINT) / "MyDrive" / WORKSPACE_NAME / "outputs" / "colab_smoke.json"
if smoke_path.exists():
    data = json.loads(smoke_path.read_text(encoding="utf-8"))
    status = "OK" if data.get("ok") else "FAILED"
    print(f"Smoke status: {status}")
    for model in data.get("models", []):
        sample = model.get("sample_file") or "n/a"
        print(
            f"- {model['repo_id']}: {model['status']} "
            f"({model.get('files_present', 0)} files, sample={sample})"
        )
else:
    print(f"No smoke artifact found at {smoke_path}. Run the helper once to generate it.")

In [ ]:
# Cell 3: Smoke-test stub for the orchestrator (safe, non-destructive)
# This cell attempts to import the orchestrator package and reports what is available.
try:
    import sparkle_motion.orchestrator as orchestrator_mod
    print('Imported sparkle_motion.orchestrator ->', orchestrator_mod)
    if hasattr(orchestrator_mod, 'Runner'):
        print('Runner class is available. You can instantiate it for a simulation run.')
        print('Example (local):')
        print("  from sparkle_motion.orchestrator import Runner")
        print("  r = Runner(run_dir='runs')")
        print("  # then use r.run(...) or similar per your orchestrator API")
    else:
        print('Runner class not found — inspect src/sparkle_motion/orchestrator.py for usage.')
except Exception as e:
    print('Could not import orchestrator module:', e)
    print('If you want to run the orchestrator, ensure the package is on PYTHONPATH (e.g., pip install -e .) or run via the repository root.')


## Optional: run a stub orchestration smoke test
This cell runs the Python runner in simulation mode (fallback adapters) so you can confirm Drive folders are writable before enabling real models.

In [ ]:
# Cell 5: Optional orchestrator smoke run (uses fallback adapters)
import importlib.util
from sparkle_motion.orchestrator import Runner
in_colab = importlib.util.find_spec("google.colab") is not None
if in_colab:
    runs_root = Path(MOUNT_POINT) / "MyDrive" / WORKSPACE_NAME / "runs"
else:
    runs_root = REPO_ROOT / "runs"
runs_root.mkdir(parents=True, exist_ok=True)
movie_plan = {
    "title": "Colab Smoke",
    "shots": [
        {
            "id": "shot_001",
            "visual_description": "Test scene",
            "duration_sec": 2.0,
            "dialogue": [{"character": "narrator", "text": "Hello from Colab"}],
        }
    ],
}
runner = Runner(runs_root=str(runs_root))
asset_refs = runner.run(movie_plan=movie_plan, run_id="colab_smoke", resume=True)
print("Smoke run complete. Final asset refs keys:", asset_refs.keys())
print("Runs directory:", runs_root)